<a href="https://colab.research.google.com/github/jx-dohwan/Aiffel_EGLMS_Project/blob/main/%5BExp_6%5DWriting_lyrics(NLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Exp_6]Writing_lyrics(NLP)

## 1. 라이브러리 버전 확인 및 import

In [11]:
import tensorflow as tf
import glob
import re
import numpy as np
from sklearn.model_selection import train_test_split


print(tf.__version__)

2.8.2


## 2. 데이터 읽어오기

In [2]:
txt_file_path = "/content/drive/MyDrive/인공지능/아이펠/ES/data/lyrics/*"

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus에 담는다.
for txt_file in txt_list:
  with open(txt_file, "r") as f:
    raw = f.read().splitlines()
    raw_corpus.extend(raw)

print('데이터 크기 : ', len(raw_corpus))
print('Examples : \n', raw_corpus[:3])

데이터 크기 :  227088
Examples : 
 ['The first words that come out', 'And I can see this song will be about you', "I can't believe that I can breathe without you"]


## 3. 데이터 정제

앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 를 권합니다.

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:

    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
#corpus[:20]



In [9]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)  
    
    print(tensor,tokenizer)
    
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    6  237 ...    0    0    0]
 [   2    8    5 ...    0    0    0]
 [   2    5   38 ...    0    0    0]
 ...
 [   2  848    1 ...    0    0    0]
 [   2  161   64 ...    3    0    0]
 [   2 4396  161 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f303196ca90>


In [6]:
print(tensor)

[[   2    6  237 ...    0    0    0]
 [   2    8    5 ...    0    0    0]
 [   2    5   38 ...    0    0    0]
 ...
 [   2  848    1 ...    0    0    0]
 [   2  161   64 ...    0    0    0]
 [   2 4396  161 ...    0    0    0]]


In [8]:
print(tokenizer.index_word)

{1: '<unk>', 2: '<start>', 3: '<end>', 4: ',', 5: 'i', 6: 'the', 7: 'you', 8: 'and', 9: 'to', 10: 'a', 11: 'it', 12: 'me', 13: 'my', 14: 'in', 15: '.', 16: 'that', 17: 's', 18: 't', 19: 'of', 20: 'on', 21: 'your', 22: 'we', 23: 'is', 24: 'm', 25: 'all', 26: 'for', 27: 'be', 28: 'like', 29: 'with', 30: 'so', 31: '?', 32: 'but', 33: 'up', 34: 'this', 35: 'what', 36: 'love', 37: 'know', 38: 'can', 39: 'no', 40: 'just', 41: 'don', 42: 'oh', 43: 'they', 44: 'got', 45: 'do', 46: '!', 47: 'when', 48: 'get', 49: 'she', 50: 'he', 51: 'if', 52: 'now', 53: 'yeah', 54: 'not', 55: 'go', 56: 'baby', 57: 'was', 58: 'out', 59: 'one', 60: 're', 61: 'll', 62: 'down', 63: 'have', 64: 'let', 65: 'her', 66: 'come', 67: 'as', 68: 'there', 69: 'see', 70: 'say', 71: 'at', 72: 'want', 73: 'from', 74: 'are', 75: 'cause', 76: 'make', 77: 'will', 78: 'time', 79: 'how', 80: 'his', 81: 'back', 82: 'never', 83: 'man', 84: 'take', 85: 'd', 86: 'girl', 87: 'here', 88: 'then', 89: 'way', 90: 'right', 91: 'more', 92: 'w

## 4. 평가 데이터셋 분리

훈련 데이터와 평가 데이터를 분리하세요!

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상 으로 설정하세요! 총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!

In [10]:
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[1])
print(tgt_input[1])

[  2   8   5  38  69  34 372  77  27 122   7   3   0   0]
[  8   5  38  69  34 372  77  27 122   7   3   0   0   0]


In [12]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=23, )

## 5.인공지능 만들기